<a href="https://colab.research.google.com/github/hawkmak3r/NLP/blob/master/NLP_assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#importing pandas library for reading our csv file
import pandas as pd
#from sklearn we import a train_test_split which helps us to divide our data of
#housing into 2 sets train and test model
from sklearn.model_selection import train_test_split
#importing numpy helps us to work with data and modify as per our need
import numpy as np
#for ploting our graphs
import matplotlib.pyplot as plt

#imports require for our network
import torch
#under are different layers we need
from torch.nn import Conv1d
from torch.nn import MaxPool1d
from torch.nn import Flatten
from torch.nn import  Linear
from torch.nn.functional import relu
from torch.utils.data import DataLoader, TensorDataset



In [0]:
#importing dataset and remove any N/A entries 
data_set = pd.read_csv('/content/housing.csv')
data_set = data_set.dropna()

#perfroming visualization
#as given in assignmet first we printed 10 records then we printed our plots 
#wwhich ar given in the assignment
#first ten records from out dataset
data_set.head(10)
#plots 
DS=data_set[:17]
plots=pd.DataFrame(DS,columns= DS.columns)
plots.plot(subplots=True, figsize=(10, 10) )



In [0]:
#for predicting medain house value column we define it as our Y variable
Y = data_set['median_house_value']
#for predicting we will use  longitude, latitude, housing median age, total 
#number of rooms, total number of bedrooms, population, number of households, 
#and median income. Defining those as X
X = data_set.loc[:,'longitude':'median_income']
print(X.shape)


(20433, 8)


In [589]:
#spliting our data 70% for training and 30% for testing while random state is
#set to 2003
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state=2003)

#to manipulate data we will convert it into numpy arrays 
#training data
x_train_np = x_train.to_numpy()
y_train_np = y_train.to_numpy()
#test data
x_test_np = x_test.to_numpy() 
y_test_np = y_test.to_numpy()
print


<function print>

In [0]:
#definning our network for predication 

class Nlp_assignment1(torch.nn.Module):
  def __init__(self, b_size, inputs, outputs):

    super(Nlp_assignment1, self).__init__()
    self.b_size = b_size
    self.inputs = inputs
    self.outputs = outputs 

    #first layer our input layer
    self.input_layer = Conv1d(inputs, b_size, 1)
    
    #creating a max pooling layer
    self.mpool_layer = MaxPool1d(1)

    # Define another convolution layer
    self.conv_layer = Conv1d(b_size , 128 , 1)

    self.flatten_layer = Flatten()

    self.lr_layer = Linear(128 , 64)
    
    #final a output layer
    self.out_layer = Linear(64, outputs)

    #All the layer are set
    #creating feeding method to give our inputs to our layer 

  def feeder(self, input):
    #as we using 1D convolution while our data is 3d we need to feed that data
    #1d way
    input = input.reshape((self.b_size, self.inputs, 1))

    output = relu(self.input_layer(input))

    output = self.mpool_layer(output)

    output = relu(self.conv_layer(output))

    output = self.flatten_layer(output)

    output = self.lr_layer(output)

    output = self.out_layer(output)

    return output

In [0]:
#block of training our model

from torch.optim import SGD
from torch.optim import Adam
from torch.nn import L1Loss
from torchvision import models

!pip install pytorch-ignite

from ignite.contrib.metrics.regression.r2_score import R2Score


b_size = 32

model = Nlp_assignment1(b_size, X.shape[1], 1)

print('The total number of trainable paramaeters are ',sum([param.nelement() for param in model.parameters()]))
model.cuda()

model = models.vgg16()
print(model)
print('as we can see from method our kernal size is 3 x 3 ')

In [0]:
import pickle

with open('0889960_1dconv_reg' ,'wb' ) as f:
  pickle.dump(model,f)


In [0]:
#Function for Average L1 loss and our R2 score 

def Avg_loss(model, data_set, train = False, optimizer = None):
  perf = L1Loss()
  score_r = R2Score()

  avg_loss, avg_score, count = 0, 0, 0
  for input, output in iter(data_set):

    predict = model.feeder(input)

    l = perf(predict, output)

    score_r.update([predict, output])
    score = score_r.compute()

    if(train):

      optimizer.zero_grad()

      l.backward()

      optimizer.step()

    avg_loss += l.item()
    avg_score += score
    count += 1
  return avg_loss / count, avg_score / count

In [0]:
epochs = 1000

# here we are using Adam becuase SGD fails to compute as per our requirement and
#Adam provides us better output then SGD
optimizer = Adam(model.parameters() , lr=1e-4)

#Again as done before we are converting data in numpy arrary so we can manipulate
#it as per our wish
inputs = torch.from_numpy(x_test_np).cuda().float()
outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0], 1)).cuda().float()
#creating a Dataloader for loading our given data for batchs 
tensor = TensorDataset(inputs , outputs)
loader = DataLoader(tensor, b_size, shuffle = True, drop_last = True)
#Starting our timer for training model
from time import perf_counter
t1_start = perf_counter()  
#intitaling our loop with given number of epochs 
for epoch in range(epochs): 
# going through the batches and get the average loss 
  avg_loss, avg_r2_score = Avg_loss(model, loader, train=True, optimizer=optimizer)
# printing average loss per one epoch as we train though them
  print("Epoch " + str(epoch + 1) + ":\n\tLoss = " + str(avg_loss) + "\n\tR^2 Score = " + str(avg_r2_score))

inputs = torch.from_numpy(x_test_np).cuda().float()
outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0],1 )).cuda().float()

tensor = TensorDataset(inputs, outputs) 
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)

avg_loss, avg_r2_score = Avg_loss(model, loader) 
print("this model has L1 loss: " + str(avg_loss)) 
print("This model has a  R^2 score of " + str(avg_r2_score))
t1_stop = perf_counter() 
  
print("Inference time in seconds ", t1_stop-t1_start)
